In [16]:
import pandas as pd 
import math
import plotly.express as px


In [17]:
sensex_1980 = "../data/SnP-Sensex-1980.csv"
sensex_df = pd.read_csv(sensex_1980, header=0)
sensex_df
del sensex_df["Low"]
del sensex_df["Open"]
del sensex_df["High"]
sensex_df["Returns"] = sensex_df["Close"].pct_change()
sensex_df= sensex_df.fillna(0)

In [18]:
sensex_df

,Month,Close,Returns
0,January-1991,982.32,0.000000
1,February-1991,1220.41,0.242375
2,March-1991,1167.97,-0.042969
3,April-1991,1235.11,0.057484
4,May-1991,1307.34,0.058481
...,...,...,...
353,June-2020,34915.80,0.076847
354,July-2020,37606.89,0.077074
355,August-2020,38628.29,0.027160
356,September-2020,38067.93,-0.014506


#### Effective Return = Return On Equity + Return on Debt - Inflated Value fo Withdrawl

In [80]:
DEBT_RETURN = 1.07
INFLATION_ANNUAL = 1.05
SWR = 4
SWR_RANGES = [1,2,3,5]
MONTHLY_DEBT_RETURN = math.pow(DEBT_RETURN, 1/12) - 1
MONTHLY_INFLATION = math.pow(INFLATION_ANNUAL, 1/12) - 1
FIRST_WITHDRAWAL = SWR/12
EQUITY_SHARE = .7
DEBT_SHARE = 1 - EQUITY_SHARE
INITIAL_NVESTMENT = 100
DATAPOINTS = len(sensex_df.index)

In [81]:
def calculate_withdrawls(initial_withdrawal):
    withdrawals = list()
    withdrawals_values = initial_withdrawal
    for i in range(DATAPOINTS):
        withdrawals_values = (1+MONTHLY_INFLATION)*withdrawals_values
        withdrawals.append(withdrawals_values)
    return withdrawals



In [82]:
def generate_portfolio(main_index, withdrawals):
    equity = INITIAL_NVESTMENT*EQUITY_SHARE
    debt = INITIAL_NVESTMENT*DEBT_SHARE
    current_month_amount = equity + debt
    monthly_portfolio_value = list()
    for index, row in sensex_df.iterrows():
        if main_index < index:
            if debt >= withdrawals[index]*DEBT_SHARE and equity >= withdrawals[index]*EQUITY_SHARE: ## Deduct from both if both are positive
                debt = (1+ MONTHLY_DEBT_RETURN)*debt - withdrawals[index]*DEBT_SHARE
                equity = (1 + row["Returns"])*equity - withdrawals[index]*EQUITY_SHARE
            elif debt < withdrawals[index]:   ## Deduct from equity if debt is less
                debt = (1+ MONTHLY_DEBT_RETURN)*debt
                equity = (1 + row["Returns"])*equity - withdrawals[index]
            elif equity < withdrawals[index]:     ## Deduct from debt if equity is less
                debt = (1+ MONTHLY_DEBT_RETURN)*debt - withdrawals[index]
                equity = (1 + row["Returns"])*equity 
            total_value = equity + debt
            monthly_portfolio_value.append(total_value)
        elif main_index == index:
            monthly_portfolio_value.append(equity + debt)
        else:
            monthly_portfolio_value.append(None)
    return monthly_portfolio_value

In [83]:
performance_at_swr = dict()
for swr in SWR_RANGES:
    initial_withdrawal = swr/12
    withdrawals = calculate_withdrawls(initial_withdrawal)
    portfolio_growth_df = pd.DataFrame()
    for main_index, main_row in sensex_df.iterrows():
        monthly_portfolio_value  = generate_portfolio(main_index, withdrawals)
        portfolio_growth_df.insert(main_index, main_row["Month"], monthly_portfolio_value)
    performance_at_swr[swr] = portfolio_growth_df

In [84]:
def inflated_value(initial_value, timeperiod, monthly_inflation):
    return initial_value * (pow((1 + monthly_inflation ), timeperiod)) 

In [85]:
## Calculate the inflated value of portfolio
inflated_corpus = list()
total_months = len(sensex_df.index)
for month  in sensex_df["Month"]:
    corpus_value = inflated_value(100, total_months, MONTHLY_INFLATION)
    inflated_corpus.append((month, corpus_value))
    total_months -= 1
inflated_corpus_df = pd.DataFrame.from_records(inflated_corpus, columns=["Month", "corpus"])
inflated_corpus_df

,Month,corpus
0,January-1991,428.694017
1,February-1991,426.954551
2,March-1991,425.222144
3,April-1991,423.496765
4,May-1991,421.778388
...,...,...
353,June-2020,102.053728
354,July-2020,101.639636
355,August-2020,101.227223
356,September-2020,100.816485


In [86]:
## Current Value of Portfolio
def get_current_value(performace_df):
    current_value_df  = performace_df.tail(1)
    current_value_df = current_value_df.transpose()
    current_value_df.columns = ["corpus"]
    current_value_df['Month'] = current_value_df.index
    current_value_df = current_value_df[['Month', 'corpus']]
    current_value_df = current_value_df.reset_index(drop=True)
    return current_value_df

In [87]:
performance_list = list()
for swr in SWR_RANGES:
    current_value_df = get_current_value(performance_at_swr[swr])
    preservation_result =  current_value_df['corpus'] - inflated_corpus_df['corpus']
    preservation_result_df = pd.DataFrame(preservation_result)
    preservation_status = preservation_result_df.apply(lambda x: True if x['corpus'] >0 else False , axis=1)
    success = len(preservation_status[preservation_status == True].index)
    performance_list.append((swr, success/len(preservation_status)))

In [88]:
performance_df = pd.DataFrame.from_records(performance_list, columns=["SWR", "Performance"])

In [89]:
performance_df

,SWR,Performance
0,1,0.784916
1,2,0.533520
2,3,0.382682
3,5,0.100559
